<a href="https://colab.research.google.com/github/drpetros11111/Hands-On-Large-Language-Models/blob/drpp_study/Copy_of_Chapter_2_Tokens_and_Token_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Chapter 2 - Tokens and Token Embeddings</h1>
<i>Exploring tokens and embeddings as an integral part of building LLMs</i>


<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961"><img src="https://img.shields.io/badge/Buy%20the%20Book!-grey?logo=amazon"></a>
<a href="https://www.oreilly.com/library/view/hands-on-large-language/9781098150952/"><img src="https://img.shields.io/badge/O'Reilly-white.svg?logo=data:image/svg%2bxml;base64,PHN2ZyB3aWR0aD0iMzQiIGhlaWdodD0iMjciIHZpZXdCb3g9IjAgMCAzNCAyNyIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPGNpcmNsZSBjeD0iMTMiIGN5PSIxNCIgcj0iMTEiIHN0cm9rZT0iI0Q0MDEwMSIgc3Ryb2tlLXdpZHRoPSI0Ii8+CjxjaXJjbGUgY3g9IjMwLjUiIGN5PSIzLjUiIHI9IjMuNSIgZmlsbD0iI0Q0MDEwMSIvPgo8L3N2Zz4K"></a>
<a href="https://github.com/HandsOnLLM/Hands-On-Large-Language-Models"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HandsOnLLM/Hands-On-Large-Language-Models/blob/main/chapter02/Chapter%202%20-%20Tokens%20and%20Token%20Embeddings.ipynb)

---

This notebook is for Chapter 2 of the [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961) book by [Jay Alammar](https://www.linkedin.com/in/jalammar) and [Maarten Grootendorst](https://www.linkedin.com/in/mgrootendorst/).

---

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961">
<img src="https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/images/book_cover.png" width="350"/></a>


### [OPTIONAL] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>

If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

---

💡 **NOTE**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

In [ ]:
# %%capture
# !pip install transformers>=4.41.2 sentence-transformers>=3.0.1 gensim>=4.3.2 scikit-learn>=1.5.0 accelerate>=0.31.0

# Downloading and Running An LLM

The first step is to load our model onto the GPU for faster inference. Note that we load the model and tokenizer separately and keep them as such so that we can explore them separately.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20
)

# Print the output
print(tokenizer.decode(generation_output[0]))

<s> Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|> Subject: My Sincere Apologies for the Gardening Mishap

Dear


In [ ]:
print(input_ids)

tensor([[    1, 14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278,
         25305,   293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,
           920,   372,  9559, 29889, 32001]], device='cuda:0')


In [ ]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

<s>
Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.
Exp
lain
how
it
happened
.
<|assistant|>


In [ ]:
generation_output

tensor([[    1, 14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278,
         25305,   293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,
           920,   372,  9559, 29889, 32001,  3323,   622, 29901,  1619,   317,
          3742,   406,  6225, 11763,   363,   278, 19906,   292,   341,   728,
           481,    13,    13, 29928,   799]], device='cuda:0')

In [ ]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


# Comparing Trained LLM Tokenizers


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [ ]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

In [ ]:
show_tokens(text, "bert-base-uncased")

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " / t / t " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [ ]:
show_tokens(text, "bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

In [ ]:
show_tokens(text, "gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

In [ ]:
show_tokens(text, "google/flan-t5-small")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

In [ ]:
# The official is `tiktoken` but this the same tokenizer on the HF platform
show_tokens(text, "Xenova/gpt-4")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

In [ ]:
# You need to request access before being able to use this tokenizer
show_tokens(text, "bigcode/starcoder2-15b")

tokenizer_config.json:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "facebook/galactica-1.3b")

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "      "  Three  t abs :   "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<s>  
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 

# Contextualized Word Embeddings From a Language Model (Like BERT)

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# Load a language model
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# Tokenize the sentence
tokens = tokenizer('Hello world', return_tensors='pt')

# Process the tokens
output = model(**tokens)[0]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

In [ ]:
output.shape

torch.Size([1, 4, 384])

In [ ]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
Hello
 world
[SEP]


In [ ]:
output

tensor([[[-3.4816,  0.0861, -0.1819,  ..., -0.0612, -0.3911,  0.3017],
         [ 0.1898,  0.3208, -0.2315,  ...,  0.3714,  0.2478,  0.8048],
         [ 0.2071,  0.5036, -0.0485,  ...,  1.2175, -0.2292,  0.8582],
         [-3.4278,  0.0645, -0.1427,  ...,  0.0658, -0.4367,  0.3834]]],
       grad_fn=<NativeLayerNormBackward0>)

# Text Embeddings (For Sentences and Whole Documents)

In [ ]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert text to text embeddings
vector = model.encode("Best movie ever!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vector.shape

(768,)

# Word Embeddings Beyond LLMs


In [ ]:
import gensim.downloader as api

# Download embeddings (66MB, glove, trained on wikipedia, vector size: 50)
# Other options include "word2vec-google-news-300"
# More options at https://github.com/RaRe-Technologies/gensim-data
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
model.most_similar([model['king']], topn=11)

[('king', 1.0000001192092896),
 ('prince', 0.8236179351806641),
 ('queen', 0.7839043140411377),
 ('ii', 0.7746230363845825),
 ('emperor', 0.7736247777938843),
 ('son', 0.766719400882721),
 ('uncle', 0.7627150416374207),
 ('kingdom', 0.7542161345481873),
 ('throne', 0.7539914846420288),
 ('brother', 0.7492411136627197),
 ('ruler', 0.7434253692626953)]

# Recommending songs by embeddings

In [1]:
import pandas as pd
from urllib import request

# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]

# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

# Building a song recommendation systems
This code downloads two datasets:

one containing playlists (lists of song IDs)

 and

another containing metadata for each song (ID, title, artist).

It then processes these datasets into data structures that can be easily used for further analysis, such as building a song recommendation system.

--------------------------------
#1. Importing Libraries:


    import pandas as pd
    from urllib import request


##import pandas as pd:
This line imports the pandas library, which is a powerful tool for data manipulation and analysis in Python.

We use the alias pd for convenience.

##from urllib import request:
This line imports the request module from the urllib library.

This module is used to open and read URLs.

------------------------------
#2. Loading Playlist Data:


##Get the playlist dataset file
    data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

## Parse the playlist dataset file. Skip the first two lines as they only contain metadata

    lines = data.read().decode("utf-8").split('\n')[2:]

## Remove playlists with only one song
    playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

These lines are responsible for downloading and processing the playlist dataset.

###request.urlopen(...):
This opens the URL containing the dataset and reads its contents.

###data.read().decode("utf-8").split('\n')[2:]:

This reads the data, decodes it as UTF-8, splits it into lines, and skips the first two lines (metadata).

###playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]:

This creates a list of playlists, where each playlist is a list of song IDs.

It filters out playlists with only one song.

---------------
#3. Loading Song Metadata:

## Load song metadata
    songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')

    songs_file = songs_file.read().decode("utf-8").split('\n')
    songs = [s.rstrip().split('\t') for s in songs_file]
    songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])

##songs_df = songs_df.set_index('id')
These lines load the song metadata (title and artist) into a pandas DataFrame.

##request.urlopen(...):

Opens and reads the URL containing song metadata.

##songs_file.read().decode("utf-8").split('\n'):

Reads, decodes, and splits the data into lines.

    songs = [s.rstrip().split('\t')

##for s in songs_file]:
Creates a list of songs, where each song is represented as a list containing its ID, title, and artist.

    s.rstrip():

This part applies the rstrip() method to the string s.

rstrip() removes any trailing whitespace characters (spaces, tabs, newlines) from the end of the string.

This ensures that there are no extra spaces or tabs at the end of the line, which could interfere with the splitting process.

    .split('\t'):

This part applies the split() method to the result of s.rstrip().

split('\t') splits the string into a list of substrings, using the tab character (\t) as the delimiter.

This is because the song metadata file uses tabs to separate the song ID, title, and artist in each line.

In simpler terms:

Imagine a line from the song metadata file looks like this:


    1234    Song Title    Artist Name\t

Here's what s.rstrip().split('\t') would do:

s.rstrip() would remove the trailing tab (\t) at the end of the line.

.split('\t') would then split the remaining string into a list using the tab characters as delimiters:

    ['1234', 'Song Title', 'Artist Name']
    
Therefore, the expression s.rstrip().split('\t') takes a line from the song metadata file, removes any trailing whitespace, and then splits it into a list of song ID, title, and artist based on the tab delimiters.

This process is essential for properly parsing the song metadata file and storing the information in a structured way for further analysis and use in the recommendation system.

##songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist']):

Creates a pandas DataFrame from the song data.

##songs_df = songs_df.set_index('id'):
Sets the 'id' column as the index of the DataFrame, making it easier to look up songs by their IDs.


In [2]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

Playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

Playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

In [3]:
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

# 1. Importing the Word2Vec class:

    from gensim.models import Word2Vec

This line imports the Word2Vec class from the gensim.models module.

gensim is a popular library for topic modeling and natural language processing tasks, and Word2Vec is a specific algorithm within it used for creating word embeddings.

----------------------
#2. Training the Word2Vec model:


    model = Word2Vec(
       playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
     )

This is where the actual training of the Word2Vec model happens.

Here's what each parameter means:

##playlists:

This is the input data for the model.

It is a list of playlists, where each playlist is itself a list of song IDs.

The model will learn to represent each song ID as a vector based on its context (i.e., the other songs it appears with in playlists).


##vector_size=32:
This specifies the dimensionality of the word embeddings.

Each song ID will be represented by a vector of 32 numbers.

Higher dimensionality can capture more complex relationships, but it also increases computational cost.

##window=20:
This defines the size of the context window. The model will consider 20 songs before and 20 songs after a given song when learning its embedding.

This means that the model considers up to 40 songs around each song to generate its context.

##negative=50:
This parameter controls the negative sampling used during training.

Negative sampling is a technique that helps the model learn better by contrasting positive examples (songs that appear together in playlists) with negative examples (songs that are randomly selected).

The value 50 here refers to the number of negative samples used for each positive sample.

##min_count=1:
This sets the minimum frequency for a song ID to be included in the vocabulary.

Song IDs that appear less than this number of times in the playlists will be ignored. Setting it to 1 ensures that every song is kept in the vocabulary.

##workers=4:
This specifies the number of worker threads to use for training.

Using multiple workers can speed up training, especially on machines with multiple cores.

In essence, this code creates and trains a Word2Vec model to learn representations of song IDs based on their co-occurrence in playlists.

The model learns to represent each song as a vector of numbers (with dimension 32) in a way that songs frequently found in the same playlist have similar vector representations.

This resulting model (stored in the model variable) is the song recommendation model, mapping from song ID to a vector representation.

In [4]:
song_id = 2172

# Ask the model for songs similar to song #2172
model.wv.most_similar(positive=str(song_id))

[('6626', 0.9979610443115234),
 ('1922', 0.9978659749031067),
 ('6685', 0.9972315430641174),
 ('2014', 0.99668288230896),
 ('2849', 0.9964174032211304),
 ('3105', 0.9961897730827332),
 ('5586', 0.9960750341415405),
 ('11473', 0.9958188533782959),
 ('1954', 0.9956790208816528),
 ('6641', 0.9955552220344543)]

In [5]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [6]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# Extract recommendations
print_recommendations(2172)

,title,artist
id,,
6626,Blackout,Scorpions
1922,One,Metallica
6685,The Trooper,Iron Maiden
2014,Youth Gone Wild,Skid Row
2849,Run To The Hills,Iron Maiden


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('title').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('artist').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['artist'].value_counts()
    for x_label, grp in _df_2.groupby('title')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('title')
_ = plt.ylabel('artist')

# Create Recommendations Lists
In summary, this code snippet defines a function that leverages the Word2Vec model to find songs similar to a given song ID.

By finding the most similar vectors to the input in the embedding space, it recommends the songs with those closest vectors.

Then it uses the recommendations to look up information about the songs from the songs_df DataFrame and return it.

The user can then use this returned information to display the recommended songs however they'd like.

Finally, the snippet demonstrates a song recommendation for a specific song ID

------------------
#1. Importing NumPy

    import numpy as np

This line imports the NumPy library, which is essential for numerical operations in Python, particularly for working with arrays. We use the alias np for convenience.

----------------------------
#2. Defining the Recommendation Function

    def print_recommendations(song_id):
       similar_songs = np.array(
         model.wv.most_similar(positive=str(song_id),topn=5)
         )[:,0]
       return  songs_df.iloc[similar_songs]

This defines a function called print_recommendations that takes a song_id as input and returns a DataFrame of recommended songs.

-----------------------------
Here's how it works:

###model.wv.most_similar(positive=str(song_id), topn=5):

This line utilizes the trained Word2Vec model (model) to find the 5 most similar songs to the input song_id.

It uses the most_similar method of the wv attribute (which represents the KeyedVectors instance holding the word embeddings).

The positive argument specifies the song ID we want recommendations for (converted to a string), and topn=5 limits the results to the top 5 most similar songs.

This call returns a list of tuples, where each tuple contains a song ID and its similarity score to the input song.

##np.array(...)[:,0]

This part converts the output of most_similar into a NumPy array and then selects only the first element (song ID) from each tuple using array slicing ([:, 0]).

This essentially creates an array containing only the IDs of the recommended songs.

##songs_df.iloc[similar_songs]:
This line uses the iloc indexer of the songs_df DataFrame (which holds the song metadata) to retrieve the rows corresponding to the recommended song IDs.

This results in a DataFrame containing information (title, artist, etc.) about the recommended songs.

##return songs_df.iloc[similar_songs]:
Finally, the function returns this DataFrame containing the recommendations.

------------------------
#3. Extracting Recommendations

    print_recommendations(2172)

This line calls the print_recommendations function with the song ID 2172.

This triggers the recommendation process for the specified song, and the function returns a DataFrame with the recommended songs for that ID. (Remember that the function itself doesn't actually print anything – it just returns the DataFrame.)



In [7]:
print_recommendations(2172)

,title,artist
id,,
6626,Blackout,Scorpions
1922,One,Metallica
6685,The Trooper,Iron Maiden
2014,Youth Gone Wild,Skid Row
2849,Run To The Hills,Iron Maiden


In [8]:
print_recommendations(842)

,title,artist
id,,
1560,In Da Club,50 Cent
27081,"Give Me Everything (w\/ Ne-Yo, Afrojack & Nayer)",Pitbull
196,I'll Be Missing You,Puff Daddy & The Family
12205,Give It Up To Me,Sean Paul
6741,Love In This Club (w\/ Young Jeezy),Usher
